### 環境設置與資料載入

In [93]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import v2
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# 檢查CUDA可用性
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 資料路徑
DATA_PATH = './train/'
class_names = sorted(os.listdir(DATA_PATH))  # 獲取30個食物類別名稱

# 建立類別到索引的映射
class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}


### 強化版資料增強

In [94]:
# 設計強化版資料增強管道
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 自定義MixUp和CutMix資料增強操作
cutmix = v2.CutMix(num_classes=len(class_names))
mixup = v2.MixUp(num_classes=len(class_names))
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

# 自定義資料增強collate函數
def collate_fn(batch):
    return cutmix_or_mixup(*torch.utils.data.default_collate(batch))


### 自定義資料集

In [95]:
class FoodDataset(Dataset):
    def __init__(self, root_dir, class_to_idx, transform=None):
        self.root_dir = root_dir
        self.class_to_idx = class_to_idx
        self.transform = transform
        self.samples = self._make_dataset()
        
    def _make_dataset(self):
        samples = []
        for class_name in sorted(os.listdir(self.root_dir)):
            class_dir = os.path.join(self.root_dir, class_name)
            if os.path.isdir(class_dir):
                for img_name in os.listdir(class_dir):
                    if img_name.endswith(('.jpg', '.jpeg', '.png')):
                        img_path = os.path.join(class_dir, img_name)
                        samples.append((img_path, self.class_to_idx[class_name]))
        return samples
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        return image, label


### MobileNetV2基礎模型設置

In [96]:
def get_mobilenetv2(num_classes=30):
    # 載入預訓練的MobileNetV2
    model = torchvision.models.mobilenet_v2(weights='IMAGENET1K_V1')
    
    # 凍結特徵提取層
    for param in model.features.parameters():
        param.requires_grad = False
    
    # 替換分類頭
    in_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(in_features, num_classes)
    )
    
    return model

### Bayesian MAML實現

In [97]:
class BayesianMAML:
    def __init__(self, model, n_way=5, k_shot=5, num_particles=10, inner_lr=0.01, meta_lr=0.001):
        self.model = model
        self.n_way = n_way
        self.k_shot = k_shot
        self.num_particles = num_particles
        self.inner_lr = inner_lr
        self.meta_optimizer = optim.Adam(model.parameters(), lr=meta_lr)
        self.loss_fn = nn.CrossEntropyLoss()
        
        # 初始化粒子集
        self.particles = self._clone_particles(model)
    
    def _clone_particles(self, model):
        """複製模型參數生成多個粒子"""
        particles = []
        for _ in range(self.num_particles):
            particle = {k: v.clone() for k, v in model.state_dict().items()}
            particles.append(particle)
        return particles
    
    def _stein_gradient(self, particles, support_x, support_y, query_x, query_y):
        """計算Stein變分梯度下降(SVGD)的梯度"""
        n = len(particles)
        gradients = []
        losses = []
        
        # 僅選擇需要梯度的參數
        trainable_params = [p for p in self.model.parameters() if p.requires_grad]
        
        for particle in particles:
            # 載入粒子參數
            self.model.load_state_dict(particle)
            
            # 內循環更新（Fast adaptation）
            self.model.train()
            support_logits = self.model(support_x)
            support_loss = self.loss_fn(support_logits, support_y)
            grads = torch.autograd.grad(support_loss, trainable_params, create_graph=True)
            
            # 在查詢集上計算損失
            query_logits = self.model(query_x)
            query_loss = self.loss_fn(query_logits, query_y)
            losses.append(query_loss)
            
            # 計算元梯度
            meta_grads = torch.autograd.grad(query_loss, trainable_params, create_graph=True)
            gradients.append(meta_grads)
        
        # 計算平均損失
        avg_loss = torch.stack(losses).mean()
        
        # 實現SVGD更新
        updated_gradients = []
        for i in range(n):
            updated_grad = []
            # 修改這裡，只對可訓練參數進行迭代
            for param_idx, param in enumerate(trainable_params):
                grad_sum = torch.zeros_like(param)
                for j in range(n):
                    # 核函數: RBF核
                    # 確保使用相同的可訓練參數
                    param_diff = torch.sum((param - trainable_params[param_idx]) ** 2)
                    h = param_diff.detach() / 0.01  # 帶寬
                    kernel = torch.exp(-h)
                    
                    # SVGD更新
                    grad_sum += kernel * gradients[j][param_idx]
                
                updated_grad.append(grad_sum / n)
            updated_gradients.append(updated_grad)
        
        return updated_gradients, avg_loss

    
    def train_step(self, support_x, support_y, query_x, query_y):
        """執行一步元學習訓練"""
        self.meta_optimizer.zero_grad()
        
        # 計算SVGD梯度
        updated_gradients, loss = self._stein_gradient(
            self.particles, support_x, support_y, query_x, query_y
        )
        
        # 只對可訓練參數更新梯度
        trainable_params = [p for p in self.model.parameters() if p.requires_grad]
        
        for param, grad in zip(trainable_params, updated_gradients[0]):
            param.grad = grad
        
        self.meta_optimizer.step()
        
        # 更新粒子集
        self.particles = self._clone_particles(self.model)
        
        return loss.item()



### 訓練流程

In [98]:
def train():
    # 載入資料集
    full_dataset = FoodDataset(DATA_PATH, class_to_idx, transform=train_transform)
    n_way = 30
    k_shot = 10
    
    # 訓練/驗證集劃分 (每類8:2)
    train_indices = []
    val_indices = []
    
    # 按類別劃分
    for class_idx in range(len(class_names)):
        class_indices = [i for i, (_, label) in enumerate(full_dataset.samples) if label == class_idx]
        np.random.shuffle(class_indices)
        split = int(0.8 * len(class_indices))
        train_indices.extend(class_indices[:split])
        val_indices.extend(class_indices[split:])
    
    # 建立訓練與驗證資料載入器
    train_loader = DataLoader(
        torch.utils.data.Subset(full_dataset, train_indices),
        batch_size=n_way*k_shot,
        shuffle=True,
        collate_fn=collate_fn  # 使用MixUp/CutMix增強
    )
    
    val_loader = DataLoader(
        torch.utils.data.Subset(full_dataset, val_indices),
        batch_size=n_way*k_shot,
        shuffle=False
    )
    
    # 初始化模型與元學習器
    model = get_mobilenetv2(num_classes=len(class_names))
    model = model.to(device)
    
    meta_learner = BayesianMAML(
        model=model,
        n_way=n_way,  # 每個元任務中的類別數
        k_shot=k_shot,  # 每類的樣本數
        num_particles=10,  # 貝葉斯粒子數
        inner_lr=0.01,
        meta_lr=0.001
    )
    
    # 訓練迴圈
    num_epochs = 50
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            # 構建元學習任務（支援集和查詢集）
            if isinstance(labels, torch.Tensor) and len(labels.shape) > 1:
                # 處理MixUp/CutMix情況
                one_hot_labels = labels
                labels = torch.argmax(one_hot_labels, dim=1)
            
            # 簡化：使用批次的前一半作為支援集，後一半作為查詢集
            n = len(images) // 2
            support_x, query_x = images[:n], images[n:]
            support_y, query_y = labels[:n], labels[n:]
            
            # 執行元學習步驟
            loss = meta_learner.train_step(support_x, support_y, query_x, query_y)
            train_loss += loss
        
        # 驗證
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = nn.CrossEntropyLoss()(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        val_accuracy = 100. * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_accuracy:.2f}%")
        
        # 保存最佳模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'model.pth')
            print("模型已保存!")


In [99]:
train()

Epoch 1/50, Train Loss: 3.4698, Val Loss: 3.4499, Val Acc: 5.00%
模型已保存!
Epoch 2/50, Train Loss: 3.4658, Val Loss: 3.3783, Val Acc: 1.67%
模型已保存!
Epoch 3/50, Train Loss: 3.4311, Val Loss: 3.3203, Val Acc: 6.67%
模型已保存!
Epoch 4/50, Train Loss: 3.3869, Val Loss: 3.3399, Val Acc: 6.67%
Epoch 5/50, Train Loss: 3.4153, Val Loss: 3.3511, Val Acc: 11.67%
Epoch 6/50, Train Loss: 3.3493, Val Loss: 3.1803, Val Acc: 20.00%
模型已保存!
Epoch 7/50, Train Loss: 3.3141, Val Loss: 3.2430, Val Acc: 11.67%
Epoch 8/50, Train Loss: 3.3320, Val Loss: 3.2238, Val Acc: 10.00%
Epoch 9/50, Train Loss: 3.2500, Val Loss: 3.2223, Val Acc: 10.00%
Epoch 10/50, Train Loss: 3.2687, Val Loss: 3.1533, Val Acc: 11.67%
模型已保存!
Epoch 11/50, Train Loss: 3.1088, Val Loss: 3.1952, Val Acc: 11.67%
Epoch 12/50, Train Loss: 3.0278, Val Loss: 3.0722, Val Acc: 18.33%
模型已保存!
Epoch 13/50, Train Loss: 3.0583, Val Loss: 3.0975, Val Acc: 18.33%
Epoch 14/50, Train Loss: 3.1498, Val Loss: 3.1421, Val Acc: 10.00%
Epoch 15/50, Train Loss: 3.0245, 